In [2]:
import requests
import defs
import pandas as pd

In [3]:
session = requests.Session()

In [17]:
ins_df = pd.read_pickle("instruments.pkl")

In [18]:
our_curr = ['EUR', 'USD', 'GBP', 'JPY','CHF', 'NZD', 'CAD', 'AUD']

In [19]:
ins_df.shape

(68, 5)

In [20]:
def fetch_candles(pair_name, count, granularity):
    url = f"{defs.OANDA_URL}/instruments/{pair_name}/candles"
    params = dict(
        count = count,
        granularity = granularity,
        price = 'MBA'
    )
    response = session.get(url, params=params, headers=defs.SECURE_HEADER)
    return response.status_code, response.json()

In [21]:
code, res = fetch_candles('EUR_USD',10,"H1")

In [22]:
code

200

In [23]:
res

{'instrument': 'EUR_USD',
 'granularity': 'H1',
 'candles': [{'complete': True,
   'volume': 4918,
   'time': '2022-05-18T18:00:00.000000000Z',
   'bid': {'o': '1.04771', 'h': '1.04847', 'l': '1.04721', 'c': '1.04733'},
   'mid': {'o': '1.04778', 'h': '1.04854', 'l': '1.04730', 'c': '1.04740'},
   'ask': {'o': '1.04785', 'h': '1.04862', 'l': '1.04737', 'c': '1.04747'}},
  {'complete': True,
   'volume': 5302,
   'time': '2022-05-18T19:00:00.000000000Z',
   'bid': {'o': '1.04735', 'h': '1.04749', 'l': '1.04639', 'c': '1.04653'},
   'mid': {'o': '1.04742', 'h': '1.04756', 'l': '1.04648', 'c': '1.04661'},
   'ask': {'o': '1.04749', 'h': '1.04762', 'l': '1.04655', 'c': '1.04669'}},
  {'complete': True,
   'volume': 2817,
   'time': '2022-05-18T20:00:00.000000000Z',
   'bid': {'o': '1.04651', 'h': '1.04662', 'l': '1.04603', 'c': '1.04616'},
   'mid': {'o': '1.04659', 'h': '1.04670', 'l': '1.04611', 'c': '1.04626'},
   'ask': {'o': '1.04667', 'h': '1.04678', 'l': '1.04619', 'c': '1.04635'}},

In [24]:
def get_candles_df(json_response):
    
    prices = ['mid','bid','ask']
    ohlc = ['o','h','l','c']
    
    our_data = []
    for candle in json_response['candles']:
        if candle['complete'] == False:
            continue
        new_dict = {}
        new_dict['time'] = candle['time']
        new_dict['volume'] = candle['volume']
        for price in prices: 
            for oh in ohlc:
                new_dict[f"{price}_{oh}"] = candle[price][oh]
        our_data.append(new_dict)
    return pd.DataFrame.from_dict(our_data)

In [25]:
def save_file(candles_df,pair,granularity):
    candles_df.to_pickle(f'his_data/{pair}_{granularity}.pkl')

In [26]:
def create_data(pair, granularity):
    code, json_data = fetch_candles( pair, 4000, granularity)
    if code != 200:
        print(pair, "Error")
        return
    df = get_candles_df(json_data)
    print(f"{pair} loaded {df.shape[0]} candles from {df.time.min()} to {df.time.max()}")
    save_file(df, pair, granularity)

In [27]:
for p1 in our_curr: 
    for p2 in our_curr:
        pair = f"{p1}_{p2}"
        if pair in ins_df.name.unique():
            create_data(pair, 'H1')
            

EUR_USD loaded 3999 candles from 2021-09-28T08:00:00.000000000Z to 2022-05-19T02:00:00.000000000Z
EUR_GBP loaded 3999 candles from 2021-09-28T05:00:00.000000000Z to 2022-05-19T02:00:00.000000000Z
EUR_JPY loaded 3999 candles from 2021-09-27T23:00:00.000000000Z to 2022-05-19T02:00:00.000000000Z
EUR_CHF loaded 3999 candles from 2021-09-28T08:00:00.000000000Z to 2022-05-19T02:00:00.000000000Z
EUR_NZD loaded 3999 candles from 2021-09-28T08:00:00.000000000Z to 2022-05-19T02:00:00.000000000Z
EUR_CAD loaded 3999 candles from 2021-09-28T08:00:00.000000000Z to 2022-05-19T02:00:00.000000000Z
EUR_AUD loaded 3999 candles from 2021-09-28T06:00:00.000000000Z to 2022-05-19T02:00:00.000000000Z
USD_JPY loaded 3999 candles from 2021-09-28T08:00:00.000000000Z to 2022-05-19T02:00:00.000000000Z
USD_CHF loaded 3999 candles from 2021-09-28T03:00:00.000000000Z to 2022-05-19T02:00:00.000000000Z
USD_CAD loaded 3999 candles from 2021-09-28T08:00:00.000000000Z to 2022-05-19T02:00:00.000000000Z
GBP_USD loaded 3999 

In [28]:
code, res = fetch_candles('EUR_USD',10,'H1' )

In [29]:
df = get_candles_df(res)

In [31]:
df.head(10)

,time,volume,mid_o,mid_h,mid_l,mid_c,bid_o,bid_h,bid_l,bid_c,ask_o,ask_h,ask_l,ask_c
0,2022-05-18T18:00:00.000000000Z,4918,1.04778,1.04854,1.04730,1.04740,1.04771,1.04847,1.04721,1.04733,1.04785,1.04862,1.04737,1.04747
1,2022-05-18T19:00:00.000000000Z,5302,1.04742,1.04756,1.04648,1.04661,1.04735,1.04749,1.04639,1.04653,1.04749,1.04762,1.04655,1.04669
2,2022-05-18T20:00:00.000000000Z,2817,1.04659,1.04670,1.04611,1.04626,1.04651,1.04662,1.04603,1.04616,1.04667,1.04678,1.04619,1.04635
3,2022-05-18T21:00:00.000000000Z,1384,1.04632,1.04671,1.04616,1.04628,1.04610,1.04631,1.04578,1.04602,1.04653,1.04721,1.04632,1.04654
4,2022-05-18T22:00:00.000000000Z,3895,1.04637,1.04752,1.04636,1.04746,1.04607,1.04743,1.04605,1.04737,1.04667,1.04760,1.04663,1.04756
5,2022-05-18T23:00:00.000000000Z,3142,1.04748,1.04752,1.04701,1.04730,1.04739,1.04743,1.04693,1.04723,1.04758,1.04762,1.04709,1.04738
6,2022-05-19T00:00:00.000000000Z,9347,1.04731,1.05006,1.04682,1.04885,1.04723,1.04997,1.04673,1.04877,1.04739,1.05015,1.04690,1.04893
7,2022-05-19T01:00:00.000000000Z,8318,1.04886,1.04912,1.04788,1.04866,1.04877,1.04902,1.04780,1.04857,1.04894,1.04921,1.04796,1.04875
8,2022-05-19T02:00:00.000000000Z,6417,1.04868,1.05012,1.04832,1.04968,1.04858,1.05003,1.04824,1.04959,1.04877,1.05021,1.04840,1.04978


In [32]:
ins_df.columns

Index(['name', 'type', 'displayName', 'pipLocation', 'marginRate'], dtype='object')

In [33]:
ins_df.name.unique()

array(['EUR_HUF', 'EUR_DKK', 'USD_MXN', 'GBP_USD', 'CAD_CHF', 'EUR_GBP',
       'GBP_CHF', 'USD_THB', 'USD_ZAR', 'EUR_NOK', 'GBP_PLN', 'EUR_HKD',
       'CHF_JPY', 'USD_SEK', 'GBP_SGD', 'GBP_HKD', 'EUR_NZD', 'SGD_CHF',
       'AUD_SGD', 'EUR_JPY', 'USD_CHF', 'USD_TRY', 'GBP_JPY', 'EUR_CZK',
       'EUR_TRY', 'USD_JPY', 'GBP_ZAR', 'SGD_JPY', 'USD_CZK', 'ZAR_JPY',
       'TRY_JPY', 'USD_DKK', 'EUR_PLN', 'AUD_CAD', 'USD_HKD', 'CAD_SGD',
       'GBP_NZD', 'CHF_ZAR', 'USD_CNH', 'NZD_USD', 'AUD_NZD', 'NZD_HKD',
       'USD_NOK', 'NZD_SGD', 'CAD_JPY', 'AUD_CHF', 'NZD_CHF', 'USD_SGD',
       'EUR_SEK', 'HKD_JPY', 'AUD_HKD', 'AUD_JPY', 'EUR_ZAR', 'EUR_AUD',
       'NZD_JPY', 'CHF_HKD', 'USD_CAD', 'EUR_CHF', 'EUR_CAD', 'USD_HUF',
       'NZD_CAD', 'EUR_SGD', 'AUD_USD', 'EUR_USD', 'GBP_AUD', 'USD_PLN',
       'CAD_HKD', 'GBP_CAD'], dtype=object)

In [34]:
eur_pk = pd.read_pickle("his_data/EUR_USD_H1.pkl")


In [35]:
eur_pk

,time,volume,mid_o,mid_h,mid_l,mid_c,bid_o,bid_h,bid_l,bid_c,ask_o,ask_h,ask_l,ask_c
0,2021-09-28T08:00:00.000000000Z,4290,1.16867,1.16890,1.16753,1.16806,1.16860,1.16884,1.16747,1.16799,1.16874,1.16897,1.16759,1.16813
1,2021-09-28T09:00:00.000000000Z,3444,1.16808,1.16850,1.16732,1.16788,1.16801,1.16843,1.16724,1.16781,1.16814,1.16857,1.16739,1.16796
2,2021-09-28T10:00:00.000000000Z,4769,1.16790,1.16834,1.16752,1.16818,1.16783,1.16828,1.16745,1.16811,1.16797,1.16842,1.16759,1.16825
3,2021-09-28T11:00:00.000000000Z,6703,1.16820,1.16934,1.16773,1.16874,1.16813,1.16927,1.16767,1.16868,1.16826,1.16941,1.16779,1.16881
4,2021-09-28T12:00:00.000000000Z,9292,1.16876,1.16890,1.16728,1.16805,1.16869,1.16883,1.16721,1.16798,1.16882,1.16896,1.16735,1.16812
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3994,2022-05-18T22:00:00.000000000Z,3895,1.04637,1.04752,1.04636,1.04746,1.04607,1.04743,1.04605,1.04737,1.04667,1.04760,1.04663,1.04756
3995,2022-05-18T23:00:00.000000000Z,3142,1.04748,1.04752,1.04701,1.04730,1.04739,1.04743,1.04693,1.04723,1.04758,1.04762,1.04709,1.04738
3996,2022-05-19T00:00:00.000000000Z,9347,1.04731,1.05006,1.04682,1.04885,1.04723,1.04997,1.04673,1.04877,1.04739,1.05015,1.04690,1.04893
3997,2022-05-19T01:00:00.000000000Z,8318,1.04886,1.04912,1.04788,1.04866,1.04877,1.04902,1.04780,1.04857,1.04894,1.04921,1.04796,1.04875
